In [241]:
import pandas as pd
import pycountry
import numpy as np
import re
import os
from dotenv import load_dotenv
import unidecode
import pycountry
from geojson import FeatureCollection, dump
import requests

In [242]:
df = pd.read_csv("../output/exports.csv")[["country", "country_of_destination", "year","annex_3", "annex_4_a", "annex_4_b", "amount"]]
df.info()
initial_len = len(df)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94025 entries, 0 to 94024
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   country                 94025 non-null  object
 1   country_of_destination  93982 non-null  object
 2   year                    94025 non-null  int64 
 3   annex_3                 61801 non-null  object
 4   annex_4_a               29088 non-null  object
 5   annex_4_b               65992 non-null  object
 6   amount                  93990 non-null  object
dtypes: int64(1), object(6)
memory usage: 5.0+ MB


/tmp/ipykernel_25147/3925779474.py:1: DtypeWarning: Columns (0,1,2,3,5,12,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../output/exports.csv")[["country", "country_of_destination", "year","annex_3", "annex_4_a", "annex_4_b", "amount"]]


In [243]:
missing = df.isna().sum()
initial len
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'missing': missing})

initial_annex_3 = int(missing_value_df.iloc[[3]].missing)

In [244]:
df

,country,country_of_destination,year,annex_3,annex_4_a,annex_4_b,amount
0,Andorra,NZ,2021,H3,NaN,"R1,R13",500.0
1,Andorra,PG,2021,"H6.1,H8,H10","D5,D9",NaN,100.0
2,Andorra,PG,2021,"H6.1,H8,H10","D5,D9",NaN,100.0
3,Andorra,NZ,2021,"H6.1,H11,H12",NaN,"R1,R13",300.0
4,Andorra,NZ,2021,"H6.1,H11,H12",D10,NaN,250.0
...,...,...,...,...,...,...,...
94020,United Kingdom of Great Britain and Northern I...,DE,2001,H4.2,NaN,R4,105.98
94021,United Kingdom of Great Britain and Northern I...,DE,2001,H4.2,NaN,R4,105.98
94022,United Kingdom of Great Britain and Northern I...,NO,2001,"H10,H11",NaN,"R3,R4",6.0
94023,Uzbekistan,KZ,2001,NaN,NaN,R2,1683.7


## Normalizing country names
### Converting country name to its alpha 2 code (https://en.wikipedia.org/wiki/ISO_3166-1)

In [245]:
def func(x):
    # -- Cleaning and formatting country name strings to get recognized by pycountry --
    # removing whitespace from beginning and end so country names get recognized properly
    temp = x.strip()
    # e.g. "Venezuela (Bolivarian Republic of) -> "Venezuela, Bolivarian Republic of"
    temp = temp.replace(" (",", ").replace(")","")
    # Côte d´Ivoire --> Côte d'Ivoire
    temp = temp.replace("´", "'")
    # Handling
    if temp == "United Kingdom of Great Britain and Northern Ireland":
        temp = "United Kingdom"
    if temp == "Türkiye":
        temp = "Turkey"
    if temp == "Swaziland":
        temp = "Eswatini"
    if temp == "Republic of Moldova":
        temp = "Moldova, Republic of"
    if temp == "Democratic Republic of the Congo" or temp == "Congo, Democratic Republic of the" or temp == "Congo, Republic of the":
        temp = "Congo, The Democratic Republic of the"
    if temp == "State of Palestine":
        temp = "Palestine, State of"
    if temp == "Bolivia":
        temp = "Bolivia, Plurinational State of"
     # -- Converting country name to alpha_2 code, which is what the other columns use --
    return pycountry.countries.get(name=temp).alpha_2.lower()

df["country"] = df["country"].apply(func)

### Checking if alpha 2 codes of other columns are correct

In [246]:
def check(x):
    if not pd.isna(x):
        temp = x.strip()
        temp = unidecode.unidecode(temp)
        temp = temp.replace("\xa0", "")
        if temp == "":
            return np.nan
        temp = "".join([ c if c.isalnum() else "" for c in temp ])
        if temp == "UK":
            temp = "GB"
        country = pycountry.countries.get(alpha_2=temp)
        if country is None:
            country = pycountry.countries.get(alpha_3=temp)
            if country is None:
                # There are still some invalid values left. We treat them as typos and therefore as nan, because we cannot infer the code that the official wanted to enter
                print(x)
                return np.nan
            else:
                return country.alpha_2.lower()
        else:
            return temp.lower()
    else:
        return np.nan

df["country_of_destination"] = df["country_of_destination"].apply(check)

A
A
A
A
A
A
HL
HL
HL
HL
HL
CS
HL
HL
HL
HL
SP
SP
F
F
GB,B
GB,B
GB,B
GB,B
GB,B
TR,D
BY,D
Eire
YU


## Cleaning code columns (annex_3, annex_4_a, annex_4_b)

In [247]:
# NEEDS CONFIRMATION
# Valid codes
unique_items_a_3 = ("H1","H3","H4.1","H4.2","H4.3","H5.1","H5.2","H6.1","H6.2","H8","H10","H11","H12","H13")
unique_items_un = ["UN1","UN3","UN4.1","UN4.2","UN4.3","UN5.1","UN5.2","UN6.1","UN6.2","UN8","UN9"]
unique_items_a_4_a = [f"D{x}" for x in range(1,17)]
unique_items_a_4_b = [f"R{x}" for x in range(1,14)]

In [248]:
def init_clean(x):
    temp = x.upper()
    temp = temp.strip()
    # deleting .0 because not part of official codes
    temp = temp.replace(".0", "")
    #sometimes someone writes e.g. H03, but 0 never part of official codes
    temp = temp.replace("0","")
    #sometimes people use whitespace to separate the letter and the number
    temp = temp.replace(" ","")
    return temp

def cleaning_codes(x, letter, unique):
    if not pd.isna(x):
        if x == "" or x == "--" or x == "-":
            return np.nan
        temp = unidecode.unidecode(x)
        # converting cell to list, because sometimes it contains more than one value, replacing other possible separators with commas
        temp = temp.replace("/", ",").replace(";",",").replace("\n",",").replace("，", ",")
        # basic cleaning
        lst = [init_clean(x) for x in temp.split(",")]
        # NEEDS CONFIRMATION
        # sometimes cell contains only a number. we are assuming they just didn't add the letter (H for example) in this case
        lst = [letter + x if letter not in x else x for x in lst]

        # NEEDS CONFIRMATION
        # sometimes a cell contains something like this: "R_". We are treating this as nan
        lst = [x for x in lst if not f"{letter}_" in x and not f"{letter}*" in x]
        if lst != []:
            return lst
        else:
            return np.nan
    else:
        return x

In [249]:
df["annex_3"] = df.apply(lambda x: cleaning_codes(x["annex_3"], "H", unique_items_a_3), axis=1)
df["annex_4_a"] = df.apply(lambda x: cleaning_codes(x["annex_4_a"], "D", unique_items_a_4_a), axis=1)
df["annex_4_b"] = df.apply(lambda x: cleaning_codes(x["annex_4_b"], "R", unique_items_a_4_b), axis=1)

There are still some invalid values left. Some are just typos.We convert them to nan because we cannot infer the code that the official wanted to enter

In [250]:
# I think we can improve cleaning
list(set(list(df[df["annex_3"].notna()]["annex_3"].explode().unique())) - set(unique_items_a_3))

['H2',
 'H4.1H12H13',
 "H'7",
 'H1AH13',
 'H3H6.1H11',
 'H8H12',
 'H4.1H4.2',
 'H6.',
 'H31',
 'H3H12',
 'H3.1',
 'H3H4.1H6.1H12H13',
 'H17',
 'H1.1',
 'HP14',
 'HXX',
 'HAZ',
 'HNR',
 'H33-35',
 'H3B',
 'H4.1H12',
 'H3H6.1',
 'HE11',
 'HE13',
 'H6.2.',
 'HE12',
 'HP15',
 'H811',
 'H6.1.H11',
 'H11YH12',
 'H4',
 'HE6.1',
 'H12H13',
 'HN13',
 'H1.8',
 'HN.A.',
 'H14',
 'HP3',
 'H3H4.1',
 'H9',
 'HNA',
 'H8.H12',
 'H3A',
 'HNOTSPECIFIED',
 'HP6',
 'HA',
 'H3H6.1H11H12H13',
 'H4.4',
 'HO(CAN)',
 'H6',
 'H3H4.1H5.1H6.1H8H11H12',
 'HNEJ',
 'H3H6.1H11H12',
 'HN12',
 'H7',
 'H6.1(H3)',
 'HP14EU',
 'H6.1(3)',
 'H8.1',
 'H4.1H5.1H6.1H8H11H12H13',
 'H4.2H4.3',
 'H6.1.',
 'H4.1H6.1',
 'H4999999999999996',
 'H4.1H6.1H12',
 'H34',
 'HN11',
 'HEJFARL',
 'H6.H8',
 'H112',
 'H3H6.1H8',
 'H6.1H',
 'H3H4.1H12',
 'H5H13',
 'HE9',
 'HETC',
 'H5',
 'H12HAZ',
 'H15',
 'H4.1.',
 'HY18',
 'H41',
 'H12.1',
 'HNOTLISTED',
 "H'6.1",
 'H61',
 'H18',
 'HE4.3',
 'H11.12',
 '1-H11',
 'H5.5',
 'HE3',
 'H4.1.H6.1',
 '

In [251]:
list(set(list(df[df["annex_4_a"].notna()]["annex_4_a"].explode().unique())) - set(unique_items_a_4_a))

['D18',
 'D',
 'DCN',
 'D19',
 'DBE',
 'D13+D1',
 'DJP',
 'DXX',
 'DZA',
 'DIL',
 'DR3',
 'DR4',
 'DLV',
 'D17',
 'DAT',
 'DE',
 'DUA',
 'DGB',
 'HD14',
 'DR5',
 'DPR',
 'DKR']

In [252]:
list(set(list(df[df["annex_4_b"].notna()]["annex_4_b"].explode().unique())) - set(unique_items_a_4_b))

['RD12', 'R14', 'RXX', 'R16', 'MIXEDR', 'R15', 'R', 'RD1']

In [253]:
def func(lst,valid):
    if isinstance(lst, list):
        temp = [x for x in lst if x in valid]
        if temp != []:
            return temp
        else:
            return np.nan
    else:
        return np.nan

In [254]:
df["annex_3"] = df.apply(lambda x: func(x["annex_3"], unique_items_a_3), axis=1)
df["annex_4_a"] = df.apply(lambda x: func(x["annex_4_a"], unique_items_a_4_a), axis=1)
df["annex_4_b"] = df.apply(lambda x: func(x["annex_4_b"], unique_items_a_4_b), axis=1)

In [255]:
df['annex_3']

0                    [H3]
1          [H6.1, H8, H1]
2          [H6.1, H8, H1]
3        [H6.1, H11, H12]
4        [H6.1, H11, H12]
               ...       
94020              [H4.2]
94021              [H4.2]
94022           [H1, H11]
94023                 NaN
94024                 NaN
Name: annex_3, Length: 94025, dtype: object

In [256]:
def un_code(h_code):
    if type(h_code)!= float:
        return [s.replace("H", "UN") if s not in ("H10", "H11", "H12", "H13") else "UN9" for s in h_code]
    else:
        return np.nan

In [257]:
df['UN_code'] = df['annex_3'].apply(un_code)

## Cleaning amount

In [258]:
def func(x):
    if not pd.isna(x):
        if isinstance(x, float):
            return x
        else:
            temp = str(x)
            # removing whitespace
            temp = temp.replace(" ", "")
            # converting , to .
            temp = temp.replace(",",".")
            # replacing all non numeric characters but .
            temp = re.sub("[^0-9.]", "", temp)
            temp = float(temp)
            return temp
    else:
        return np.nan

df["amount"] = df.apply(lambda x: func(x["amount"]), axis=1)

## Cleaning year

In [259]:
def func(x):
    if not pd.isna(x):
        return int(x)
    else:
        return np.nan

df["year"] = df.apply(lambda x: func(x["year"]), axis=1)

## Dealing with missing values

We are going to drop all rows that have missing values for amount and country_of_destination.

In [260]:
df[df["country"]=="NA"] = np.nan
df = df[df['amount'].notna()]
df = df[df['country_of_destination'].notna()]
df = df[df['country'].notna()]

In [261]:
print(f"Lost {initial_len - len(df)} rows")

Lost 84 rows


In [332]:
missing = df.isna().sum()
missing_value_df = pd.DataFrame({'column_name': df.columns,
                                 'missing': missing})

print(f" after processing {int(missing_value_df.iloc[[3]].missing) - initial_annex_3} less rows of annex_3. Initial value: {initial_annex_3}")

 after processing 6630 less rows of annex_3. Initial value: 32224


## Creating separate columns for codes

In [263]:
df[unique_items_un + ["multiple", "unspecified"]] = 0
for index, row in df.iterrows():
    if type(row["UN_code"]) != float:
        if len(row["UN_code"]) == 1:
            df.loc[index, row["UN_code"][0]] = row["amount"]
        else:
            df.loc[index, "multiple"] = row["amount"]
    else:
        df.loc[index, "unspecified"] = row["amount"]


## Renaming colums

In [264]:
df.rename(columns={"country": "origin", "country_of_destination": "destination"}, inplace=True)

In [265]:
df

,origin,destination,year,annex_3,annex_4_a,annex_4_b,amount,UN_code,UN1,UN3,...,UN4.2,UN4.3,UN5.1,UN5.2,UN6.1,UN6.2,UN8,UN9,multiple,unspecified
0,ad,nz,2021.0,[H3],NaN,"[R1, R13]",500.00,[UN3],0.0,500.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,ad,pg,2021.0,"[H6.1, H8, H1]","[D5, D9]",NaN,100.00,"[UN6.1, UN8, UN1]",0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
2,ad,pg,2021.0,"[H6.1, H8, H1]","[D5, D9]",NaN,100.00,"[UN6.1, UN8, UN1]",0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0.0
3,ad,nz,2021.0,"[H6.1, H11, H12]",NaN,"[R1, R13]",300.00,"[UN6.1, UN9, UN9]",0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,300.0,0.0
4,ad,nz,2021.0,"[H6.1, H11, H12]",[D1],NaN,250.00,"[UN6.1, UN9, UN9]",0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,250.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94020,gb,de,2001.0,[H4.2],NaN,[R4],105.98,[UN4.2],0.0,0.0,...,105.98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94021,gb,de,2001.0,[H4.2],NaN,[R4],105.98,[UN4.2],0.0,0.0,...,105.98,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
94022,gb,no,2001.0,"[H1, H11]",NaN,"[R3, R4]",6.00,"[UN1, UN9]",0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0
94023,uz,kz,2001.0,NaN,NaN,[R2],1683.70,NaN,0.0,0.0,...,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1683.7


## Aggregating


In [280]:
cols = ["origin","destination","year"]
agg_functions = {"UN1" : "sum" ,"UN3": "sum","UN4.1": "sum","UN4.2": "sum","UN4.3": "sum","UN5.1": "sum","UN5.2": "sum","UN6.1": "sum","UN6.2": "sum","UN8": "sum","UN9": "sum", "unspecified": "sum", "multiple": "sum"}
df_new = df.groupby(cols, dropna=False).aggregate(agg_functions).reset_index()

In [281]:
df_new

,origin,destination,year,UN1,UN3,UN4.1,UN4.2,UN4.3,UN5.1,UN5.2,UN6.1,UN6.2,UN8,UN9,unspecified,multiple
0,ad,es,2002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7777.400,0.00
1,ad,es,2003.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,486.800,0.00
2,ad,es,2004.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,425.980,0.00
3,ad,es,2005.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,621.950,0.00
4,ad,es,2006.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1044.092,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8121,za,se,2015.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1000.0,0.000,0.00
8122,za,sg,2010.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,720.0,0.000,0.00
8123,za,tr,2018.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,21.24
8124,zm,fi,2002.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,235.000,0.00


In [291]:
len(df_new[(df_new["unspecified"]!=0 and (df_new[[unique_items_un]] == 0).any(axis=1))])

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [331]:
print(f"{len(df_new[(df_new.iloc[:,3:14] == 0).all(axis=1)])} of {len(df_new)} rows have an amount of zero everywhere but unspecified or multiple")
print(f"{len(df_new[(df_new.iloc[:,14:] == 0).all(axis=1)])} of {len(df_new)} rows have no amount on unspecified and multiple")
print(f"{len(df_new[(df_new.iloc[:,14:15] != 0).all(axis=1)])} of {len(df_new)} rows have an amount on unspecified")
print(f"{len(df_new[(df_new.iloc[:,15:16] != 0).all(axis=1)])} of {len(df_new)} rows have an amount on multiple")

3584 of 8126 rows have an amount of zero everywhere but unspecified or multiple
2368 of 8126 rows have no amount on unspecified and multiple
4286 of 8126 rows have an amount on unspecified
2099 of 8126 rows have an amount on multiple


In [330]:
df_new[(df_new.iloc[:,15:16] != 0).all(axis=1)]

,origin,destination,year,UN1,UN3,UN4.1,UN4.2,UN4.3,UN5.1,UN5.2,UN6.1,UN6.2,UN8,UN9,unspecified,multiple
14,ad,es,2016.0,0.0,745.272,0.0,0.0,0.0,0.0,0.0,0.0,2.420,193.200,1813.486,205.565,65.992
15,ad,es,2017.0,0.0,553.091,0.0,0.0,0.0,0.0,0.0,0.0,2.910,178.720,1778.233,24.367,58.424
16,ad,es,2018.0,0.0,311.053,0.0,0.0,0.0,0.0,0.0,0.0,2.450,169.630,2276.018,33.420,87.723
17,ad,es,2019.0,0.0,146.551,0.0,0.0,0.0,0.0,0.0,0.0,1.956,193.660,2302.386,9.263,83.732
18,ad,es,2020.0,0.0,115.578,0.0,0.0,0.0,0.0,0.0,0.0,2.400,224.624,1928.386,5.737,85.755
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8097,za,gb,2020.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,10.0,0.000,0.000,0.000,0.000,150.000
8106,za,kp,2018.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,45000.000
8107,za,kr,2007.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,5000.000
8109,za,kr,2014.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,18200.000,2500.000,5000.000


In [ ]:
df_new.to_csv("../output/processed/clean.csv", index=False)

In [268]:
# # Source: https://towardsdatascience.com/dealing-with-list-values-in-pandas-dataframes-a177e534f173
# def boolean_df(item_lists, unique_items):# Create empty dict
#     bool_dict = {}
#
#     # Loop through all the tags
#     for i, item in enumerate(unique_items):
#         # Apply boolean mask
#         bool_dict[item] = item_lists.apply(lambda x: item in x)
#
#     # Return the results as a dataframe
#     return pd.DataFrame(bool_dict)

In [269]:
# df["annex_3"].notna()

In [270]:
# df_bool_h = boolean_df(df[df["annex_3"].notna()]["annex_3"], unique_items_a_3)
# df_bool_d = boolean_df(df[df['annex_4_a'].notna()]["annex_4_a"], unique_items_a_4_a)
# df_bool_r = boolean_df(df[df["annex_4_b"].notna()]["annex_4_b"], unique_items_a_4_b)
# df_bool_un = boolean_df(df[df["UN_code"].notna()]["UN_code"], unique_items_un)

In [271]:
# df_bool_h

In [272]:
# df_bool_un

In [273]:
# # only using un code for now
# df = pd.concat([df[["country", "country_of_destination", "year", "amount"]], df_bool_un], axis=1)
# df.reset_index(drop=True, inplace=True)
# # df[df.columns.intersection([*unique_items_a_3, *unique_items_a_4_a, *unique_items_a_4_b])] = df[df.columns.intersection([*unique_items_a_3, *unique_items_a_4_a, *unique_items_a_4_b])].fillna(np.nan)

## Geocoding

In [274]:
# load_dotenv()
# TOKEN=os.getenv("MAPBOX_TOKEN")

In [275]:
# headers = {'Accept': 'application/json'}
# dct = {}
# countries = list(df.country.unique())
# for i in countries:
#     url = f"https://api.mapbox.com/geocoding/v5/mapbox.places/{i}.json?&types=country&access_token={TOKEN}"
#     r = requests.get(url)
#     jason = r.json()
#     dct[i] = jason["features"][0]["center"]


In [276]:
# def func(x):
#     lat = dct[x][0]
#     lon = dct[x][1]
#     return lat,lon
#
#
# df["origin_lat"],df["origin_lon"] = df.apply(lambda x: func(x["country"]), axis=1)
# df["destination_lat"],df["destination_lon"] = df.apply(lambda x: func(x["country_of_destination"]), axis=1)

In [277]:
# for i in countries:
#     df.loc[df["country"] == i, "lat_origin"]=dct[i][1]
#     df.loc[df["country"] == i, "lon_origin"]=dct[i][0]
#     df.loc[df["country_of_destination"] == i, "lat_destination"]=dct[i][1]
#     df.loc[df["country_of_destination"] == i, "lon_destination"]=dct[i][0]

## Renaming columns

In [278]:
# df

In [279]:
# df.to_csv("../output/processed/clean.csv", index=False)